In [107]:
import cv2
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import pandas as pd
from PIL import Image

In [ ]:
# Génération de données d'entraînement avec augmentation

datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [109]:
# Callbacks 
 
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True),
    ModelCheckpoint(filepath='model.keras', save_best_only=True),
]

In [ ]:
def generate_batch(image):
    train_data = []
    train_labels = []
    image_normalized = image.astype('float32') / 255.0
    image_reshaped = np.expand_dims(image_normalized, axis=-1)
    image_batch = np.expand_dims(image_reshaped, axis=0)
    
    # Générer 10 images augmentées par image originale
    augmented_count = 0
    for batch in datagen.flow(image_batch, batch_size=1):
        if augmented_count >= 10:  # Limiter à 10 images augmentées par image
            break
        
        augmented_image = batch[0]
        augmented_image = (augmented_image.squeeze() * 255).astype('uint8')
        
        train_data.append(augmented_image)
        train_labels.append(1)
        augmented_count += 1

    return train_data, train_labels


In [ ]:
# Charger les données d'entraienement 

def load_data():
    # Initialiser les listes pour les données et les labels
    train_data = []
    train_labels = []
    test_data = []
    test_labels = []

    # Données oliwer
    for photos in ['oliwer']:
        train = f'processed/{photos}/train/' # Images d'entraînement

        for file in os.listdir(train):
            if file.endswith('.jpg') or file.endswith('.png'):
                img = os.path.join(train, file) # Lire le chemin de l'image
                image = cv2.imread(img, cv2.IMREAD_GRAYSCALE) # Transformer l'image en gris
                image = cv2.resize(image, (96, 96)) # Redimensionner l'image à 96x96 pixels
                
                # Ajouter l'image originale
                train_data.append(image)
                train_labels.append(1) # 1 = Oliwer
                
                # Générer des images augmentées
                augmented_data, augmented_labels = generate_batch(image)
                train_data.extend(augmented_data)
                train_labels.extend(augmented_labels)

        for file in os.listdir(f'processed/{photos}/test/'):
            if file.endswith('.jpg') or file.endswith('.png'):
                img = os.path.join(f'processed/{photos}/test/', file) # Lire le chemin de l'image de test
                image = cv2.imread(img, cv2.IMREAD_GRAYSCALE) # Transformer l'image en gris
                image = cv2.resize(image, (96, 96)) # Redimensionner l'image à 64x64 pixels
                test_data.append(image) # Ajouter l'image à la liste des données de test
                test_labels.append(1) # Ajouter le label correspondant

    # Données aléatoires
    random_train_dir = './processed/non_oliwer/train'  # Dossier avec des images aléatoires d'entraînement
    random_test_dir = './processed/non_oliwer/test'    # Dossier avec des images aléatoires de test
    
    # Traitement des images d'entraînement aléatoires (limité à 240)
    random_train_files = [f for f in os.listdir(random_train_dir) if f.endswith('.jpg') or f.endswith('.png')]
    for file in random_train_files[:2400]:
        img = os.path.join(random_train_dir, file)
        image = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (96, 96))
        train_data.append(image)  # Ajout aux données d'entraînement
        train_labels.append(0)    # Label 0 pour non-Oliwer

    # Traitement des images de test aléatoires (limité à 60)
    random_test_files = [f for f in os.listdir(random_test_dir) if f.endswith('.jpg') or f.endswith('.png')]
    for file in random_test_files[:600]:
        img = os.path.join(random_test_dir, file)
        image = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (96, 96))
        test_data.append(image)   # Ajout aux données de test
        test_labels.append(0)     # Label 0 pour non-Oliwer

    return np.array(train_data), np.array(train_labels), np.array(test_data), np.array(test_labels)

In [ ]:
# Préparation des données 

train_data, train_labels, test_data, test_labels = load_data() # Charger les données

# Afficher le nombre d'échantillons dans chaque ensemble
print(f"Nombre d'images d'entraînement: {len(train_data)}")
print(f"Nombre de labels d'entraînement: {len(train_labels)}") 
print(f"Nombre d'images de test: {len(test_data)}")
print(f"Nombre de labels de test: {len(test_labels)}")

# Vérifier la cohérence des tailles
assert len(train_data) == len(train_labels), "Incohérence entre données et labels d'entraînement"
assert len(test_data) == len(test_labels), "Incohérence entre données et labels de test"

# Normalisation des données
train_data = train_data.astype('float32') / 255.0
test_data = test_data.astype('float32') / 255.0

# Transformer les images en échelle de gris
train_data = np.expand_dims(train_data, axis=-1)
test_data = np.expand_dims(test_data, axis=-1)


Nombre d'images d'entraînement: 5040
Nombre de labels d'entraînement: 5040
Nombre d'images de test: 660
Nombre de labels de test: 660


In [ ]:
# Modèle
# Conv2D = Convolution 2D: Couche de convolution 2D pour extraire les caractéristiques des images
# MaxPooling2D = Max Pooling 2D: Réduit la dimension des caractéristiques extraites
# Flatten = Aplatir: Transforme les données 2D en un vecteur 1D pour la couche dense
# Dense = Couche dense: Couche entièrement connectée pour la classification
# Dropout = Abandon: Technique de régularisation pour éviter le surapprentissage

model = Sequential(
    [
        Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)),
        MaxPooling2D(2, 2),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Dropout(0.25),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),   # Dropout plus élevé avant la sortie
        Dense(1, activation='sigmoid')
    ]
)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 


In [ ]:
# Entraîner le modèle

history = model.fit(
    train_data, 
    train_labels,
    validation_data=(test_data, test_labels),
    epochs=50,
    batch_size=32,
    callbacks=callbacks
)

model.save('model.keras')

Epoch 1/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 45ms/step - accuracy: 0.8948 - loss: 0.2493 - val_accuracy: 0.9576 - val_loss: 0.0966 - learning_rate: 0.0010
Epoch 2/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step - accuracy: 0.9813 - loss: 0.0480 - val_accuracy: 0.9985 - val_loss: 0.0112 - learning_rate: 0.0010
Epoch 3/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step - accuracy: 0.9877 - loss: 0.0391 - val_accuracy: 0.9939 - val_loss: 0.0148 - learning_rate: 0.0010
Epoch 4/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 42ms/step - accuracy: 0.9938 - loss: 0.0162 - val_accuracy: 0.9939 - val_loss: 0.0144 - learning_rate: 0.0010
Epoch 5/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 42ms/step - accuracy: 0.9952 - loss: 0.0104 - val_accuracy: 0.9985 - val_loss: 0.0059 - learning_rate: 0.0010
Epoch 6/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 42ms/step - accuracy: 0.9964 - loss: 0.0088 - val_accuracy: 0.9970 - val_loss: 0.0035 - learning_rate: 0.0010
Epoch 7/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 42ms/step - accuracy: 0.9984 - loss: 0.

In [ ]:
# Évaluation du modèle

val = model.evaluate(test_data, test_labels)

load_model = keras.models.load_model('model.keras')
evaluation = load_model.evaluate(test_data, test_labels)

image = cv2.imread('github.png', cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (100, 100))

image = image.astype('float32') / 255.0
image = np.expand_dims(image, axis=-1)  
image = np.expand_dims(image, axis=0)   

prediction = load_model.predict(image)
confidence = prediction[0][0]

if confidence > 0.5:
    print(f"C'est Oliwer avec {confidence:.2%} de confiance")
else:
    print(f"Ce n'est pas Oliwer avec {(1-confidence):.2%} de confiance")

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9985 - loss: 0.0022
Évaluation du modèle: [0.002228757832199335, 0.9984848499298096]
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9985 - loss: 0.0022 
Évaluation du modèle chargé: [0.002228757832199335, 0.9984848499298096]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Prédiction: [[1.2473721e-30]]
Confiance: 0.00%
Ce n'est pas Oliwer avec 100.00% de confiance
